## Preprocessing

In [61]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [62]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [97]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        38
USE_CASE               5
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [98]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [99]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 0
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [66]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [67]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts[classification_counts > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [100]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 5
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
Other       45
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C4100        6
C1720        6
C2400        6
C1257        5
C1600        5
Name: CLASSIFICATION, dtype: int64

In [101]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy_application_df = pd.get_dummies(application_df)
dummy_application_df.head(5)

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,USE_CASE_ProductDev,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,1,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [102]:
# Split our preprocessed data into our features and target arrays
X = dummy_application_df.drop('IS_SUCCESSFUL', axis=1)
y = dummy_application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [71]:
print(y.value_counts())

1    18261
0    16038
Name: IS_SUCCESSFUL, dtype: int64


In [72]:
X.shape

(34299, 83)

In [73]:
X_train.shape

(25724, 83)

In [74]:
X_test.shape

(8575, 83)

In [103]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [104]:
X_train_scaled.shape

(25724, 77)

## Compile, Train and Evaluate the Model

In [105]:
# Define the model
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=X_train_scaled.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 80)                6240      
                                                                 
 dense_10 (Dense)            (None, 30)                2430      
                                                                 
 dense_11 (Dense)            (None, 1)                 31        
                                                                 
Total params: 8,701
Trainable params: 8,701
Non-trainable params: 0
_________________________________________________________________


In [106]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [110]:
# Train the model
history = nn.fit(X_train_scaled, y_train, epochs=50, verbose=2, batch_size=X_train_scaled.shape[1])

Epoch 1/50
335/335 - 3s - loss: 0.5530 - accuracy: 0.7303 - 3s/epoch - 8ms/step
Epoch 2/50
335/335 - 1s - loss: 0.5531 - accuracy: 0.7307 - 1s/epoch - 4ms/step
Epoch 3/50
335/335 - 1s - loss: 0.5532 - accuracy: 0.7299 - 1s/epoch - 4ms/step
Epoch 4/50
335/335 - 1s - loss: 0.5526 - accuracy: 0.7309 - 672ms/epoch - 2ms/step
Epoch 5/50
335/335 - 1s - loss: 0.5527 - accuracy: 0.7308 - 641ms/epoch - 2ms/step
Epoch 6/50
335/335 - 1s - loss: 0.5516 - accuracy: 0.7311 - 644ms/epoch - 2ms/step
Epoch 7/50
335/335 - 1s - loss: 0.5517 - accuracy: 0.7316 - 700ms/epoch - 2ms/step
Epoch 8/50
335/335 - 1s - loss: 0.5508 - accuracy: 0.7319 - 677ms/epoch - 2ms/step
Epoch 9/50
335/335 - 1s - loss: 0.5508 - accuracy: 0.7312 - 602ms/epoch - 2ms/step
Epoch 10/50
335/335 - 1s - loss: 0.5507 - accuracy: 0.7313 - 650ms/epoch - 2ms/step
Epoch 11/50
335/335 - 1s - loss: 0.5513 - accuracy: 0.7309 - 634ms/epoch - 2ms/step
Epoch 12/50
335/335 - 1s - loss: 0.5501 - accuracy: 0.7315 - 602ms/epoch - 2ms/step
Epoch 13/5

In [108]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=1, batch_size=1)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 [==============================] - 15s 2ms/step - loss: 0.5605 - accuracy: 0.7250
Loss: 0.5604611039161682, Accuracy: 0.7250145673751831


In [109]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity.h5')

## Optimization Method 1 - Add/Drop Columns

In [22]:
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_squared_error

In [51]:
# Create a Lasso model
lasso_model = Lasso(alpha=0.1)  # Adjust the alpha parameter for regularization strength
lasso_model.fit(X_train, y_train)

lasso_predictions = lasso_model.predict(X_test)
lasso_rmse = mean_squared_error(y_test, lasso_predictions, squared=False)
print("Lasso RMSE:", lasso_rmse)

# Create a Ridge model
ridge_model = Ridge(alpha=0.1)  # Adjust the alpha parameter for regularization strength
ridge_model.fit(X_train, y_train)

ridge_predictions = ridge_model.predict(X_test)
ridge_rmse = mean_squared_error(y_test, ridge_predictions, squared=False)
print("Ridge RMSE:", ridge_rmse)

Lasso RMSE: 0.49911599627875314
Ridge RMSE: 0.4413887184457818


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=4.66388e-22): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


In [95]:
lasso_coeffs = lasso_model.coef_
ridge_coeffs = ridge_model.coef_
coefficients_df = pd.DataFrame(columns=["Column", "Lasso Coefficient", "Ridge Coefficient", "Drop_Potential"])

for i in range(len(lasso_coeffs)):
    column_name = X.columns[i]
    lasso_coefficient = '{:.14f}'.format(lasso_coeffs[i])
    ridge_coefficient = '{:.14f}'.format(ridge_coeffs[i])
    if lasso_coeffs[i] == 0 and ((ridge_coeffs[i] > -0.1 and ridge_coeffs[i] <= 0) or (ridge_coeffs[i] < 0.1 and ridge_coeffs[i] >= 0)):
        drop = 1
    else:
        drop = 0
    row_data = {"Column": column_name, "Lasso Coefficient": lasso_coefficient, "Ridge Coefficient": ridge_coefficient, "Drop_Potential": drop}
    coefficients_df = pd.concat([coefficients_df, pd.DataFrame(row_data, index=[0])])

pd.set_option('display.max_rows', None)
print(len(coefficients_df[coefficients_df['Drop_Potential'] == 1]))
coefficients_df

39


,Column,Lasso Coefficient,Ridge Coefficient,Drop_Potential
0,STATUS,-0.00000000000000,-0.12527893244370,0
0,ASK_AMT,-0.00000000005140,-0.00000000000461,0
0,APPLICATION_TYPE_T10,0.00000000000000,0.25570371764943,0
0,APPLICATION_TYPE_T12,0.00000000000000,0.04879671483683,1
0,APPLICATION_TYPE_T13,0.00000000000000,-0.08808563967589,1
0,APPLICATION_TYPE_T14,0.00000000000000,0.46955139438807,0
0,APPLICATION_TYPE_T15,0.00000000000000,0.18783760833367,0
0,APPLICATION_TYPE_T17,0.00000000000000,0.05107022726380,1
0,APPLICATION_TYPE_T19,-0.00000000000000,-0.40099581227983,0
0,APPLICATION_TYPE_T2,-0.00000000000000,-0.36137203649957,0


In [96]:
# Expriment and drop columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['SPECIAL_CONSIDERATIONS', 'ORGANIZATION'], axis=1)

In [71]:
cutoff_value = 50
application_types_to_replace = working_op_df['APPLICATION_TYPE'].value_counts()[working_op_df['APPLICATION_TYPE'].value_counts() < cutoff_value].index.tolist()

for app in application_types_to_replace:
    working_op_df['APPLICATION_TYPE'] = working_op_df['APPLICATION_TYPE'].replace(app,"Other")

In [72]:
cutoff_value = 100
classifications_to_replace = working_op_df['CLASSIFICATION'].value_counts()[working_op_df['CLASSIFICATION'].value_counts() < cutoff_value].index.tolist()

for cls in classifications_to_replace:
    working_op_df['CLASSIFICATION'] = working_op_df['CLASSIFICATION'].replace(cls,"Other")

In [73]:
dummy_working_op_df = pd.get_dummies(working_op_df)
dummy_working_op_df.head(5)

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,108590,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,6692,1,0,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,142590,1,0,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [74]:
X1 = dummy_working_op_df.drop('IS_SUCCESSFUL', axis=1)
y1 = dummy_working_op_df['IS_SUCCESSFUL']

X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.25, random_state=1)

scaler = StandardScaler()

X_scaler1 = scaler.fit(X_train1)

X_train_scaled1 = X_scaler1.transform(X_train1)
X_test_scaled1 = X_scaler1.transform(X_test1)

In [75]:
# Define the model
nn_op1 = tf.keras.models.Sequential()

# First hidden layer
nn_op1.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=X_train_scaled1.shape[1]))

# Second hidden layer
nn_op1.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Output layer
nn_op1.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_op1.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 80)                3520      
                                                                 
 dense_13 (Dense)            (None, 30)                2430      
                                                                 
 dense_14 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [76]:
nn_op1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history1 = nn_op1.fit(X_train_scaled1, y_train1, epochs=100, verbose=2, batch_size=X_train_scaled1.shape[1])

Epoch 1/100
599/599 - 2s - loss: 0.5716 - accuracy: 0.7192 - 2s/epoch - 3ms/step
Epoch 2/100
599/599 - 1s - loss: 0.5550 - accuracy: 0.7294 - 1s/epoch - 2ms/step
Epoch 3/100
599/599 - 2s - loss: 0.5526 - accuracy: 0.7308 - 2s/epoch - 3ms/step
Epoch 4/100
599/599 - 2s - loss: 0.5511 - accuracy: 0.7305 - 2s/epoch - 4ms/step
Epoch 5/100
599/599 - 3s - loss: 0.5490 - accuracy: 0.7331 - 3s/epoch - 6ms/step
Epoch 6/100
599/599 - 2s - loss: 0.5480 - accuracy: 0.7313 - 2s/epoch - 3ms/step
Epoch 7/100
599/599 - 1s - loss: 0.5475 - accuracy: 0.7319 - 964ms/epoch - 2ms/step
Epoch 8/100
599/599 - 1s - loss: 0.5469 - accuracy: 0.7345 - 972ms/epoch - 2ms/step
Epoch 9/100
599/599 - 2s - loss: 0.5461 - accuracy: 0.7330 - 2s/epoch - 3ms/step
Epoch 10/100
599/599 - 1s - loss: 0.5466 - accuracy: 0.7349 - 1000ms/epoch - 2ms/step
Epoch 11/100
599/599 - 1s - loss: 0.5458 - accuracy: 0.7340 - 957ms/epoch - 2ms/step
Epoch 12/100
599/599 - 1s - loss: 0.5445 - accuracy: 0.7344 - 976ms/epoch - 2ms/step
Epoch 13/

## Optimization Method 2 - Altering Model

In [ ]:
# Model creation function for ease of coding later
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid', 'leaky_relu', 'elu', 'selu'])

    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=120,
        step=5), activation=activation, input_dim=X_train.shape[1]))
    
    for i in range(hp.Int('num_layers', 2, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=120,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [ ]:
# Install & import the kerastuner library
!pip install keras_tuner
import keras_tuner as kt

In [ ]:
tuner = kt.Hyperband(
    create_model,
    objective="accuracy",
    max_epochs=5,
    hyperband_iterations=2)

In [ ]:
tuner.search(X_train_scaled,y_train,epochs=5,validation_data=(X_test_scaled,y_test), batch_size=X_train_scaled.shape[1])

In [ ]:
top_hyper = tuner.get_best_hyperparameters(5)
for param in top_hyper:
    print(param.values)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_op = tf.keras.models.Sequential()

# Take optimized hyperparams and make a model
nn_op.add(tf.keras.layers.Dense(units=param.values['first_units'], activation=param.values['activation'], input_dim=X_train_scaled.shape[1]))
for i in range(param.values['num_layers']):
    nn_op.add(tf.keras.layers.Dense(units=param.values[f'units_{i}'], activation=param.values['activation']))

# Output layer
nn_op.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_op.summary()

In [ ]:
# Compile the model
nn_op.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
history = nn_op.fit(X_train_scaled, y_train, epochs=100, verbose=1, batch_size=X_train_scaled.shape[1])

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_op.evaluate(X_test_scaled,y_test,verbose=1, batch_size=1)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
nn_op.save('AlphabetSoupCharity_Optimization.h5')